<a href="https://colab.research.google.com/github/JustHawker/CycleGAN-Keras/blob/master/NLP_lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install nltk
!pip install pandas
!pip install sklearn

In [29]:
import numpy as np
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [30]:
df = pd.read_csv('IMDB Dataset.csv')

In [31]:
X_list = df.review.to_list()[:5000]
Y = df.sentiment[:5000]

In [32]:
tfidf_vectorizer = TfidfVectorizer(max_features = 5000)
X_tfidf = tfidf_vectorizer.fit_transform(X_list)

count_vectorizer = CountVectorizer(max_features = 5000)
X_count = count_vectorizer.fit_transform(X_list)

ngram_vectorizer = CountVectorizer(ngram_range=(1,3), max_features = 5000)
X_ngram = ngram_vectorizer.fit_transform(X_list)

In [33]:
svc = SVC()
svc_params = {'C':np.linspace(0.1, 1.0, 5),
             'kernel': ['linear', 'poly', 'rbf'],
             'gamma': ['scale', 'auto']}

rfc = RandomForestClassifier()
rfc_params = {'n_estimators': list(range(5,10)), 
              'criterion':['entropy','gini'],
              'max_depth':list(range(3,6)), 
              'max_features':list(range(2,20,2))}

In [34]:
params_scores= []
i=0
for classifier, param, clf_name in zip([svc, rfc], [svc_params, rfc_params], ['SVC', 'Random forest']):
    for X, name in zip([X_tfidf, X_count, X_ngram], ['tf-idf','word count','n-gram']):
        clf = GridSearchCV(classifier, param_grid=param, cv=3)
        clf.fit(X,Y)
        params_scores.append((clf_name, name, clf.best_params_, clf.best_score_))
        i+=1
        print('model trained: {}/6'.format(i))

model trained: 1/6
model trained: 2/6
model trained: 3/6
model trained: 4/6
model trained: 5/6
model trained: 6/6


In [36]:
t = sorted(params_scores, key=lambda tup: tup[3])[-1]
print('лучший результат {} по итогам кросс-валидации показывает модель {} c векторизацией через {}.\nОптимальные параметры {}'.format(t[3],t[0], t[1], t[2]))

лучший результат 0.8588025492140469 по итогам кросс-валидации показывает модель SVC c векторизацией через tf-idf.
Оптимальные параметры {'C': 1.0, 'gamma': 'scale', 'kernel': 'rbf'}


In [37]:
#все реультаты
params_scores

[('SVC',
  'tf-idf',
  {'C': 1.0, 'gamma': 'scale', 'kernel': 'rbf'},
  0.8588025492140469),
 ('SVC',
  'word count',
  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'},
  0.8324003866693648),
 ('SVC',
  'n-gram',
  {'C': 0.1, 'gamma': 'scale', 'kernel': 'linear'},
  0.8186032661414896),
 ('Random forest',
  'tf-idf',
  {'criterion': 'gini', 'max_depth': 5, 'max_features': 14, 'n_estimators': 8},
  0.6622069343634274),
 ('Random forest',
  'word count',
  {'criterion': 'gini', 'max_depth': 5, 'max_features': 18, 'n_estimators': 8},
  0.6723988935706257),
 ('Random forest',
  'n-gram',
  {'criterion': 'entropy',
   'max_depth': 5,
   'max_features': 18,
   'n_estimators': 9},
  0.6808023749391778)]

Выполним обучение модели с наилучшим результатом на полном наборе с оптимальными параметрами

In [41]:
best = SVC(**t[2])

In [42]:
from sklearn.model_selection import cross_val_score

In [43]:
X_list = df.review.to_list()
Y = df.sentiment

tfidf_vectorizer = TfidfVectorizer(max_features = 5000)
X_tfidf = tfidf_vectorizer.fit_transform(X_list)

In [45]:
clf = cross_val_score(SVC(**t[2]), X_tfidf, Y, cv=3)

In [46]:
clf

array([0.89584208, 0.89074219, 0.89355574])